In [1]:
#Python libraries
from pyomo.environ import *
# Pyomo library for solving a problem
from pyomo.opt import SolverFactory
# Pyomo library for checking the solver statuse 
from pyomo.opt import SolverStatus, TerminationCondition
#panda is a useful data manipulation library
import numpy as np
import pandas as pd
import networkx as nx
import pylab as plt
from scipy import sparse
from sys import getsizeof

In [2]:
#Homemade functions
from getParam import getParam
from getDemand import getDemand
from objRule import objRule
from flowBalRule import flowBalRule
from flowLinkRule import flowLinkRule
from costLinkRule import costLinkRule


filenameNet = "Data/SiouxFalls/SiouxFalls_net.tntp"

param,A,M,N,L = getParam(filenameNet)

filenameTrip = "Data/SiouxFalls/SiouxFalls_trips.tntp"

d, OD_pair = getDemand(filenameTrip,L)

In [3]:
model=ConcreteModel()
# model.links=Set(initialize = (a[0]+1 for a in A))
model.links=RangeSet(0,len(A)-1,1)
model.od=RangeSet(0,M.shape[1]-1,1)
model.nodes=RangeSet(0,M.shape[0]-1,1)
model.f=Var(model.links,within=NonNegativeReals)
model.fk=Var(model.links,model.od,within=NonNegativeReals)
model.c=Var(model.links,within=NonNegativeReals)
# model.t0 = Param(initialize=param['t_0'])

In [4]:
model.obj = Objective(expr=objRule(model))

In [ ]:
model.pprint()

In [5]:
model.flowBal =  ConstraintList()
for i in model.nodes:
    for k in model.od:
        model.flowBal.add(flowBalRule(model,d,M,i,k))
#Constraint(model.nodes,model.od, expr=flowBalRule(model,d,M,i,k) for i in model.nodes for k in model.od)

In [6]:
model.flowLink =  ConstraintList()
model.costLink = ConstraintList()
for a in model.links:
    model.flowLink.add(flowLinkRule(model,a))
    model.costLink.add(costLinkRule(model,param,a))
#model.flowLink = Constraint(model.links, rule=flowLinkRule)

In [ ]:
model.pprint()

In [7]:
opt = SolverFactory("ipopt")

#opt.options["mipgap"] = 0.05
#store the results 
results = opt.solve(model)
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1976
  Number of variables: 5928
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 36.27587056159973
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
model.display()

Model unknown

  Variables:
    f : Size=76, Index=links
        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
          0 :     0 :     3099.9999984922024 :  None : False : False : NonNegativeReals
          1 :     0 :      6299.999999637267 :  None : False : False : NonNegativeReals
          2 :     0 :      599.9999988519211 :  None : False : False : NonNegativeReals
          3 :     0 :      6299.999998904751 :  None : False : False : NonNegativeReals
          4 :     0 :      699.9999992776346 :  None : False : False : NonNegativeReals
          5 :     0 :      4627.876780460055 :  None : False : False : NonNegativeReals
          6 :     0 :     4072.1232184633905 :  None : False : False : NonNegativeReals
          7 :     0 :      699.9999985651693 :  None : False : False : NonNegativeReals
          8 :     0 :      5599.999998435979 :  None : False : False : NonNegativeReals
          9 :     0 :      827.8767810784057 :  None : False : False : No

        (10, 41) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 42) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 43) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 44) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 45) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 46) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 47) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 48) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 49) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 50) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (10, 51) :     0 :                    0.0 :  None : False : Fa

        (50, 11) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 12) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 13) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 14) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 15) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 16) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 17) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 18) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 19) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 20) :     0 :                    0.0 :  None : False : False : NonNegativeReals
        (50, 21) :     0 :                    0.0 :  None : False : Fa

fk
